---

**Import python libraries and set display options**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pyarrow
import numpy as np
from scipy.stats import zscore
    
# Set pandas display options to show full column content
pd.set_option('display.max_colwidth', None)

# Set pandas display options to show all columns
pd.set_option('display.max_columns', None)

---

**Import the raw data**

In [2]:
# Read the raw CSV data file and save it as a parquet file (this step can be skipped if you alread have it saved as parquet) 
#df_raw = pd.read_csv('Raw_data/datakit.csv')
#df_raw.to_parquet('Raw_data/datakit.parquet')

In [3]:
# Read the parquet data file
df_raw = pd.read_parquet('Raw_data/datakit.parquet')

---

**Clean the data**

In [4]:
# Since most of the data points in the gender and dob columns are missing, we will remove these 4 columns.
df_1 = df_raw.drop(columns=['question_user_gender', 'question_user_dob', 'response_user_gender', 'response_user_dob'])

In [5]:
# Delete rows where question_topic is missing
df_1 = df_1.dropna(subset=['question_topic'])

In [6]:
# Delete the response topic column
df_1 = df_1.drop(columns=['response_topic'])

In [7]:
# Delete duplicate rows
df_1.drop_duplicates(inplace=True)

In [8]:
# Remove question_id/question_topic/response_user_id combos that appear more than once (this is to remove potentially incorrect data: Noticed several instances where there are many responses from one responder to the same question, but judging from the response content, most of these answers are not related to the given question at all)

# Count occurrences of each combo
combo_counts = df_1.groupby(['question_id', 'question_topic', 'response_user_id']).size()

# Get pairs that appear exactly once
combo_to_keep = combo_counts[combo_counts == 1].index

# Filter df_1 to keep only those pairs
df_1 = df_1.set_index(['question_id', 'question_topic', 'response_user_id']).loc[combo_to_keep].reset_index()

In [9]:
# Reset index to avoid gaps
df_1 = df_1.reset_index(drop=True)

---

**Create summary table of leadership metrics (by question_topic)**

In [10]:
def compute_summary_metrics(df_1):
    g = df_1.groupby(['response_user_id', 'question_topic', 'response_user_country_code'])
    out = g.agg(
    N_questions_answered=('question_id', 'nunique'),
    N_users_served=('question_user_id', 'nunique'),
    ).reset_index()
    return out

In [11]:
df_1_sum = compute_summary_metrics(df_1)

In [12]:
# Create a weighted leadership combo score, applying Z-score normalization to N_questions_answered and N_users_served
df_1_sum['Leadership_score'] = (0.5 * zscore(df_1_sum['N_questions_answered']) +
                                  0.5 * zscore(df_1_sum['N_users_served']))

In [13]:
# Keep only rows where N_questions_answered is >= 5
df_1_sum = df_1_sum[df_1_sum['N_questions_answered'] >= 5].reset_index(drop=True)

In [14]:
# Sort by Leadership_score
df_1_sum.sort_values('Leadership_score', ascending=False)

,response_user_id,question_topic,response_user_country_code,N_questions_answered,N_users_served,Leadership_score
135098,810401,maize,ke,7030,5112,462.635451
246725,1254217,chicken,ke,3809,2774,250.721841
267665,1362926,chicken,ug,3396,2733,233.900111
217513,1151861,maize,ke,2591,2235,184.390505
268112,1364605,cattle,ke,2330,1966,164.037345
...,...,...,...,...,...,...
488055,3689260,cotton,ke,5,1,-0.020921
165434,950938,butternut-squash,ke,5,1,-0.020921
119515,726825,passion-fruit,ke,5,1,-0.020921
162205,943270,duck,ke,5,1,-0.020921


In [15]:
# Save the data for use in dashboard
df_1_sum.to_csv('Raw_data/Most_frequent_responders_by_question_topic.csv')

---

**Group the question topics**

In [16]:
# Summary of question_topics (You can skip this step and use "Challenge 3_Community Leaders/JesperJayaswal/q_topics.csv")
#df_q_topic = (df_1['question_topic'].value_counts(normalize=True,dropna=True) * 100).round(1)
#df_q_topic.to_csv('q_topics.csv')# Manually update this file and assign a question topic group to each question_topic, and then save it again

In [17]:
# Update df_1 with the summary table of question topic groups
df_q_topic= pd.read_csv('q_topics.csv')
df_2 = df_1.merge(df_q_topic, on=['question_topic'], how='left')

In [18]:
# % summary of question_topics groups
df_q_topic_groups = (df_2['question_topic_group'].value_counts(normalize=True,dropna=True) * 100).round(1)
df_q_topic_groups

question_topic_group
Crop         60.2
Livestock    38.2
Other         1.1
fish          0.5
Name: proportion, dtype: float64

---

**Additional data cleaning**

In [19]:
# Delete the question topic column
df_2 = df_2.drop(columns=['question_topic'])

In [20]:
# Delete duplicate rows
df_2.drop_duplicates(inplace=True)

In [21]:
# Reset index to avoid gaps
df_2 = df_2.reset_index(drop=True)

---

**Create summary table of leadership metrics (by question_topic_Group)**

In [22]:
def compute_summary_metrics(df_2):
    g = df_2.groupby(['response_user_id', 'question_topic_group', 'response_user_country_code'])
    out = g.agg(
    N_questions_answered=('question_id', 'nunique'),
    N_users_served=('question_user_id', 'nunique'),
    ).reset_index()
    return out

In [23]:
df_2_sum = compute_summary_metrics(df_2)

In [24]:
# Create a leadership combo score

# Calculate weighted Z-score normalized metric
df_2_sum['Leadership_score'] = (0.5 * zscore(df_2_sum['N_questions_answered']) +
                                  0.5 * zscore(df_2_sum['N_users_served']))

# Sort by Leadership_score
df_2_sum.sort_values('Leadership_score', ascending=False)

,response_user_id,question_topic_group,response_user_country_code,N_questions_answered,N_users_served,Leadership_score
256221,810401,Crop,ke,10087,7190,223.032477
470002,1364605,Crop,ke,7496,5838,172.508053
344966,1049966,Livestock,ke,6687,5649,159.949028
470003,1364605,Livestock,ke,6586,5078,150.829022
344965,1049966,Crop,ke,5632,4806,135.341264
...,...,...,...,...,...,...
1263684,3826655,Livestock,ke,1,1,-0.216460
1263683,3826655,Crop,ke,1,1,-0.216460
14,24,Other,ke,1,1,-0.216460
8,19,Other,ke,1,1,-0.216460


In [25]:
# Keep only rows where N_questions_answered is >= 5
df_2_sum = df_2_sum[df_2_sum['N_questions_answered'] >= 5].reset_index(drop=True)

In [26]:
# Save the data for use in dashboard
df_2_sum.to_csv('Raw_data/Most_frequent_responders_by_question_topic_group.csv')